## YELP API Data Scraping 

In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import json


In [2]:
from yelp.client import Client, json, Business 

In [3]:
api_key='HGrgvWVfOb8BmRHDJ5ux-V1y-WbAX81JGFGQ2aF2x9Ko9BFMdSsaG1lsvA7w8bAM1gOCf4NZjhKQ0a8tsvs7OhvekIZnpbfk4A1orAOg9q3enXOZsJOnzctclwW2XXYx'
headers = {'Authorization': 'Bearer %s' % api_key}

In [4]:
url  = 'https://api.yelp.com/v3/businesses/search'
params = {'categories': ['firedepartments', 
                         'policedepartments',
                        'municipality',
                        'jailsandprisons',
                        'postoffices',
                        'courthouses'],
             'limit': 50,
             'radius': 40000,
             'location':'San Diego'}

In [5]:
# Making a get request to the API
req=requests.get(url, params=params, headers=headers)
# proceed only if the status code is 200
print('The status code is {}'.format(req.status_code))

The status code is 200


In [6]:
json.loads(req.text);

{'businesses': [{'id': 'Nm4fWKvAffuKouPudLxYzQ',
   'alias': 'hall-of-justice-san-diego',
   'name': 'Hall of Justice',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/B8AqRdocdSWI_4QEvX69FA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/hall-of-justice-san-diego?adjust_creative=RlZoBlEC3sxNugOYPhwapg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=RlZoBlEC3sxNugOYPhwapg',
   'review_count': 138,
   'categories': [{'alias': 'courthouses', 'title': 'Courthouses'}],
   'rating': 3.0,
   'coordinates': {'latitude': 32.716255, 'longitude': -117.166046},
   'transactions': [],
   'location': {'address1': '330 W Broadway',
    'address2': '',
    'address3': '',
    'city': 'San Diego',
    'zip_code': '92101',
    'country': 'US',
    'state': 'CA',
    'display_address': ['330 W Broadway', 'San Diego, CA 92101']},
   'phone': '',
   'display_phone': '',
   'distance': 8348.459368415044},
  {'id': 'Tg8Oq1ws_JaKFK6j_hzVyQ',
   'alias': 'us-p

In [7]:
data = req.json()

In [8]:
# There are 17 'businesses in lifeline 1 in San Diego
len(data['businesses'])

50

In [9]:
# Looping through what we just got from the API 
names = []       
categories = []
addresses = []
lat = []
long = []

for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

In [10]:
# Using the offset function to get through the next 50
url  = 'https://api.yelp.com/v3/businesses/search'
params = {'categories': ['firedepartments', 
                         'policedepartments',
                        'municipality',
                        'jailsandprisons',
                        'postoffices',
                        'courthouses'],
             'limit': 50,
              'offset': 50,
             'radius': 40000,
             'location':'San Diego'}

# Making a get request to the API
req=requests.get(url, params=params, headers=headers)
data = req.json()
print(len(data['businesses'])) # if less that 50, stop searching

for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])
    
print(len(names))


7
57


In [11]:
# Making a Lifeline 1 Dataframe
df1 = pd.DataFrame()
df1['business'] = names
df1['category'] = categories
df1['address'] = addresses
df1['lat'] = lat
df1['long'] = long
df1['lifeline'] = 1 # Creating a column for lifeline for when we merge later

In [12]:
df1['address'] = df1['address'].map(lambda x: str(x)[:-2])
df1['address'] = df1['address'].map(lambda x: str(x)[2:])
df1

,business,category,address,lat,long,lifeline
0,Hall of Justice,courthouses,"330 W Broadway', 'San Diego, CA 92101",32.716255,-117.166046,1
1,US Post Office,postoffices,"3288 Adams Ave', 'San Diego, CA 92116",32.763660,-117.123210,1
2,US Post Office,postoffices,"8650 Genesee Ave', 'Ste 214', 'San Diego, CA 9...",32.868689,-117.215112,1
3,US Post Office,postoffices,"11251 Rancho Carmel Dr', 'San Diego, CA 92199",32.983006,-117.081886,1
4,US Post Office,postoffices,"Macy's Home Furniture"", '1640 Camino Del Rio N...",32.769165,-117.148858,1
5,US Post Office,postoffices,"3223 Greyling Dr', 'San Diego, CA 92123",32.801380,-117.139890,1
6,Superior Court of California,courthouses,"1100 Union St', 'San Diego, CA 92101",32.716974,-117.165605,1
7,US Post Office,postoffices,"3911 Cleveland Ave', 'San Diego, CA 92103",32.749420,-117.150630,1
8,US Post Office,postoffices,"51 Horton Plz', 'San Diego, CA 92101",32.715401,-117.162423,1
9,US Post Office,postoffices,"3298 Governor Dr', 'San Diego, CA 92122",32.851190,-117.214457,1


In [85]:
df1.to_csv('Lifeline_1.csv')

### Lifeline 2 - Food, Water, Shelter

In [13]:
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['foodbanks',
                            'animalshelters',
                            'homelessshelters',
                            'communitycenters',
                            'waterstores',
                            'grocery',
                            'wholesale'
                           ],
             'limit': 50,
             'radius': 40000,
             'location': 'San Diego',
            }

In [14]:
req = requests.get(url=endpoint, headers=headers, params=parameter)
req.status_code

200

In [15]:
data = req.json()
len(data['businesses'])

50

In [16]:
# Looping through what we just got from the API 
names = []       
categories = []
addresses = []
lat = []
long = []

for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

In [17]:
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['foodbanks',
                            'animalshelters',
                            'homelessshelters',
                            'communitycenters',
                            'waterstores',
                            'grocery',
                            'wholesale'
                           ],
             'limit': 50,
             'offset': 50,
             'radius': 40000,
             'location': 'San Diego',
            }

req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses'])) # if less that 50, stop searching

for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])
    
print(len(names))

50
100


In [18]:
# Offset 100
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['foodbanks',
                            'animalshelters',
                            'homelessshelters',
                            'communitycenters',
                            'waterstores',
                            'grocery',
                            'wholesale'
                           ],
             'limit': 50,
             'offset': 100,
             'radius': 40000,
             'location': 'San Diego',
            }

req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses'])) # if less that 50, stop searching

for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])
    
print(len(names))

50
150


In [19]:
# Offset 150
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['foodbanks',
                            'animalshelters',
                            'homelessshelters',
                            'communitycenters',
                            'waterstores',
                            'grocery',
                            'wholesale'
                           ],
             'limit': 50,
             'offset': 150,
             'radius': 40000,
             'location': 'San Diego',
            }

req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses'])) # if less that 50, stop searching

for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])
    
print(len(names))

50
200


In [20]:
# Offset 200
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['foodbanks',
                            'animalshelters',
                            'homelessshelters',
                            'communitycenters',
                            'waterstores',
                            'grocery',
                            'wholesale'
                           ],
             'limit': 50,
             'offset': 200,
             'radius': 40000,
             'location': 'San Diego',
            }

req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses'])) # if less that 50, stop searching

for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])
    
print(len(names))

50
250


In [21]:
# Offset 250
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['foodbanks',
                            'animalshelters',
                            'homelessshelters',
                            'communitycenters',
                            'waterstores',
                            'grocery',
                            'wholesale'
                           ],
             'limit': 50,
             'offset': 250,
             'radius': 40000,
             'location': 'San Diego',
            }

req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses'])) # if less that 50, stop searching

for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])
    
print(len(names))

45
295


In [22]:
# Making a Lifeline 2 Dataframe
df2 = pd.DataFrame()
df2['business'] = names
df2['category'] = categories
df2['address'] = addresses
df2['lat'] = lat
df2['long'] = long
df2['lifeline'] = 2 # Creating a column for lifeline for when we merge later

In [23]:
#Cleaning up the address column
df2['address'] = df2['address'].map(lambda x: str(x)[:-2])
df2['address'] = df2['address'].map(lambda x: str(x)[2:])
df2

,business,category,address,lat,long,lifeline
0,Mona Lisa Italian Foods,grocery,"2061 India St', 'San Diego, CA 92101",32.726220,-117.169050,2
1,The Marketplace,delis,"2601 5th Ave', 'San Diego, CA 92103",32.732680,-117.160000,2
2,Trader Joe's,grocery,"1072 Camino Del Rio N', 'San Diego, CA 92108",32.766165,-117.155166,2
3,Pancho Villa's Farmer's Market,grocery,"3245 El Cajon Blvd', 'San Diego, CA 92104",32.754780,-117.123600,2
4,Zion Market,grocery,"7655 Clairemont Mesa Blvd', 'San Diego, CA 92111",32.831369,-117.155172,2
...,...,...,...,...,...,...
290,Focas-Friends of County Animal Shelters,animalshelters,"San Diego, CA 92101",32.721100,-117.164310,2
291,Oscars Market,grocery,"4210 Market St', 'San Diego, CA 92102",32.711312,-117.104477,2
292,Jerry's Mkt,grocery,"841 S 45th St', 'San Diego, CA 92113",32.697121,-117.097725,2
293,Discount Depot,grocery,"3403 E Plaza Blvd', 'National City, CA 91950",32.684630,-117.071340,2


In [86]:
df2.to_csv('Lifeline_2.csv')

### Lifeline 3 -Health and Medical

In [24]:
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['emergencymedicine',
                            'emergencyrooms',
                            'hospitals',
                            'medcenters',
                           'urgent_care',
                           'assistedliving'],
             'limit': 50,
             'radius': 40000,
             'location': 'San Diego',
            }

In [25]:
req= requests.get(url=endpoint, headers=headers, params=parameter)
req.status_code

200

In [26]:
data = req.json()
len(data['businesses'])

50

In [27]:
# Looping through what we just got from the API 
names = []       
categories = []
addresses = []
lat = []
long = []

for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

In [28]:
#Using the offset parameter to get more businesses
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['emergencymedicine',
                            'emergencyrooms',
                            'hospitals',
                            'medcenters',
                           'urgent_care',
                           'assistedliving'],
             'offset': 50,
             'limit': 50,
             'radius': 40000,
             'location': 'San Diego',
            }

In [29]:
req = requests.get(url=endpoint, headers=headers, params=parameter)

In [30]:
data = req.json()
len(data['businesses'])

50

In [31]:
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

In [32]:
# Making sure to not write over the previous 50
len(names)

100

In [33]:
# Offset to 100
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['emergencymedicine',
                            'emergencyrooms',
                            'hospitals',
                            'medcenters',
                           'urgent_care',
                           'assistedliving'],
             'offset': 100,
             'limit': 50,
             'radius': 40000,
             'location': 'San Diego',
            }

In [34]:
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
len(data['businesses'])

50

In [35]:
# Looping through the next 50
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

In [36]:
len(names)

150

In [37]:
# Offset to 150
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['emergencymedicine',
                            'emergencyrooms',
                            'hospitals',
                            'medcenters',
                           'urgent_care',
                           'assistedliving'],
             'offset': 150,
             'limit': 50,
             'radius': 40000,
             'location': 'San Diego',
            }

In [38]:
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
len(data['businesses'])

50

In [39]:
# Looping through the next 50
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

In [40]:
len(names)

200

In [41]:
# Offset to 200
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['emergencymedicine',
                            'emergencyrooms',
                            'hospitals',
                            'medcenters',
                           'urgent_care',
                           'assistedliving'],
             'offset': 200,
             'limit': 50,
             'radius': 40000,
             'location': 'San Diego',
            }
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
len(data['businesses'])

50

In [42]:
# Looping through the next 50
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])
len(names)

250

In [43]:
# Offset to 250
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['emergencymedicine',
                            'emergencyrooms',
                            'hospitals',
                            'medcenters',
                           'urgent_care',
                           'assistedliving'],
             'limit': 50,
             'radius': 40000,
             'location': 'San Diego',
            }
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
len(data['businesses'])

50

In [44]:
# Looping through the next 50
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])
len(names)

300

In [45]:
# Offset to 300
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['emergencymedicine',
                            'emergencyrooms',
                            'hospitals',
                            'medcenters',
                           'urgent_care',
                           'assistedliving'],
             'offset': 300,
             'limit': 50,
             'radius': 40000,
             'location': 'San Diego',
            }
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

# Looping through the next 50
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

29
329


In [46]:
# Making a Lifeline 3 Dataframe
df3 = pd.DataFrame()
df3['business'] = names
df3['category'] = categories
df3['address'] = addresses
df3['lat'] = lat
df3['long'] = long
df3['lifeline'] = 3 # Creating a column for lifeline for when we merge later

In [47]:
df3['address'] = df3['address'].map(lambda x: str(x)[:-2])
df3['address'] = df3['address'].map(lambda x: str(x)[2:])
df3.head()

,business,category,address,lat,long,lifeline
0,Sharp Rees-Stealy Downtown,medcenters,"300 Fir St', 'San Diego, CA 92101",32.725331,-117.161677,3
1,Marque Urgent Care,urgent_care,"4490 Fanuel St', 'San Diego, CA 92109",32.798310,-117.246440,3
2,Sharp Mary Birch Hospital for Women & Newborns,hospitals,"3003 Health Center Dr', 'San Diego, CA 92123",32.798620,-117.155051,3
3,PlushCare Urgent Care,urgent_care,"San Diego, CA 92109",32.795220,-117.237660,3
4,AFC Urgent Care,urgent_care,"5671 Balboa Ave', 'Ste 100', 'San Diego, CA 92111",32.820086,-117.179035,3


In [87]:
df3.to_csv('Lifeline_3.csv')

## Lifeline 4 - Energy (Fuel and Power)

In [48]:
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['fueldocks',
                            'servicestations',
                            'utilities',
                            'electricitysuppliers',
                            'evchargingstations'
                            'naturalgassuppliers',
                            'watersuppliers'],
             'limit': 50,
             'radius': 40000,
             'location': 'San Diego',
            }

In [49]:
req = requests.get(url=endpoint, headers=headers, params=parameter)
req.status_code

200

In [50]:
data = req.json()
len(data['businesses'])

50

In [51]:
names = []       
categories = []
addresses = []
lat = []
long = []

for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

50


In [52]:
# Setting the offset to 50
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['fueldocks',
                            'servicestations',
                            'utilities',
                            'electricitysuppliers',
                            'evchargingstations'
                            'naturalgassuppliers',
                            'watersuppliers'],
             'limit': 50,
            'offset': 50,
             'radius': 40000,
             'location': 'San Diego',
            }
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

# Looping through the next 50
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

50
100


In [53]:
# Setting the offset to 100
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['fueldocks',
                            'servicestations',
                            'utilities',
                            'electricitysuppliers',
                            'evchargingstations'
                            'naturalgassuppliers',
                            'watersuppliers'],
             'limit': 50,
            'offset': 100,
             'radius': 40000,
             'location': 'San Diego',
            }
# Calling on the API
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

# Looping through
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

50
150


In [54]:
# Setting the offset to 150
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['fueldocks',
                            'servicestations',
                            'utilities',
                            'electricitysuppliers',
                            'evchargingstations'
                            'naturalgassuppliers',
                            'watersuppliers'],
             'limit': 50,
            'offset': 150,
             'radius': 40000,
             'location': 'San Diego',
            }
# Calling on the API
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

# Looping through
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

50
200


In [55]:
# Setting the offset to 200
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['fueldocks',
                            'servicestations',
                            'utilities',
                            'electricitysuppliers',
                            'evchargingstations'
                            'naturalgassuppliers',
                            'watersuppliers'],
             'limit': 50,
            'offset': 200,
             'radius': 40000,
             'location': 'San Diego',
            }
# Calling on the API
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

# Looping through
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

48
248


In [56]:
# Making a Lifeline 4 Dataframe
df4 = pd.DataFrame()
df4['business'] = names
df4['category'] = categories
df4['address'] = addresses
df4['lat'] = lat
df4['long'] = long
df4['lifeline'] = 4 # Creating a column for lifeline for when we merge later

In [57]:
df4['address'] = df4['address'].map(lambda x: str(x)[:-2])
df4['address'] = df4['address'].map(lambda x: str(x)[2:])
df4.head()

,business,category,address,lat,long,lifeline
0,Costco,servicestations,"4605 Morena Blvd', 'San Diego, CA 92117",32.822250,-117.225270,4
1,Hazard Center Auto Mart,servicestations,"7698 Friars Rd', 'San Diego, CA 92108",32.772730,-117.158000,4
2,Costco Gas,servicestations,"2345 Fenton Pkwy', 'San Diego, CA 92108",32.780333,-117.130112,4
3,PB European & Imports,autorepair,"1885 Garnet Ave', 'San Diego, CA 92109",32.800650,-117.235610,4
4,Hillcrest 76,servicestations,"3795 6th Ave', 'San Diego, CA 92103",32.746716,-117.159203,4


In [88]:
df4.to_csv('Lifeline_4.csv')

## Lifeline 5 - Communication

In [58]:
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['telecommunications',
                            'printmedia',
                            'radiostations',
                            'televisionstations',
                           'isps'],
             'limit': 50,
             'radius': 40000,
             'location': 'San Diego',
            }

In [59]:
# Calling on the API
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

50


In [60]:
names = []       
categories = []
addresses = []
lat = []
long = []

for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

50


In [61]:
# Setting the offset to 50
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['telecommunications',
                            'printmedia',
                            'radiostations',
                            'televisionstations',
                           'isps'],
             'limit': 50,
            'offset': 50,
             'radius': 40000,
             'location': 'San Diego',
            }
# Calling on the API
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

# Looping through
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

50
100


In [62]:
# Setting the offset to 100
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['telecommunications',
                            'printmedia',
                            'radiostations',
                            'televisionstations',
                           'isps'],
             'limit': 50,
            'offset': 100,
             'radius': 40000,
             'location': 'San Diego',
            }
# Calling on the API
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

# Looping through
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

50
150


In [63]:
# Setting the offset to 150
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['telecommunications',
                            'printmedia',
                            'radiostations',
                            'televisionstations',
                           'isps'],
             'limit': 50,
            'offset': 150,
             'radius': 40000,
             'location': 'San Diego',
            }
# Calling on the API
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

# Looping through
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

50
200


In [64]:
# Setting the offset to 200
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['telecommunications',
                            'printmedia',
                            'radiostations',
                            'televisionstations',
                           'isps'],
             'limit': 50,
            'offset': 200,
             'radius': 40000,
             'location': 'San Diego',
            }
# Calling on the API
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

# Looping through
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

19
219


In [67]:
# Making a Lifeline 5 Dataframe
df5 = pd.DataFrame()
df5['business'] = names
df5['category'] = categories
df5['address'] = addresses
df5['lat'] = lat
df5['long'] = long
df5['lifeline'] = 5 # Creating a column for lifeline for when we merge later

In [68]:
df5['address'] = df5['address'].map(lambda x: str(x)[:-2])
df5['address'] = df5['address'].map(lambda x: str(x)[2:])
df5.head()

,business,category,address,lat,long,lifeline
0,T-Mobile,mobilephones,"7608 Linda Vista Rd', 'Ste 120', 'San Diego, C...",32.801498,-117.158535,5
1,Webpass,isps,"1360 5th Ave', 'San Diego, CA 92101",32.719440,-117.160324,5
2,KSDS Jazz 88.3,radiostations,"1313 Park Blvd', 'San Diego, CA 92101",32.717458,-117.152670,5
3,Ego id Media - San Diego,graphicdesign,"3538 Adams Ave', 'San Diego, CA 92116",32.763610,-117.116390,5
4,ZTelco,isps,"9105 Chesapeake Dr', 'San Diego, CA 92123",32.839190,-117.132500,5


In [89]:
df5.to_csv('Lifeline_5.csv')

## Lifeline 6 - Transportation

In [70]:
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['transport'],
             'limit': 50,
             'radius': 40000,
             'location': 'San Diego',
            }

In [71]:
# Calling on the API
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

50


In [72]:
names = []       
categories = []
addresses = []
lat = []
long = []

for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

50


In [73]:
# Setting the offset to 50
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['telecommunications',
                            'printmedia',
                            'radiostations',
                            'televisionstations',
                           'isps'],
             'limit': 50,
            'offset': 50,
             'radius': 40000,
             'location': 'San Diego',
            }
# Calling on the API
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

# Looping through
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

50
100


In [74]:
# Setting the offset to 100
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['telecommunications',
                            'printmedia',
                            'radiostations',
                            'televisionstations',
                           'isps'],
             'limit': 50,
            'offset': 100,
             'radius': 40000,
             'location': 'San Diego',
            }
# Calling on the API
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

# Looping through
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

50
150


In [75]:
# Setting the offset to 150
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['telecommunications',
                            'printmedia',
                            'radiostations',
                            'televisionstations',
                           'isps'],
             'limit': 50,
            'offset': 150,
             'radius': 40000,
             'location': 'San Diego',
            }
# Calling on the API
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

# Looping through
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

50
200


In [76]:
# Setting the offset to 200
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['telecommunications',
                            'printmedia',
                            'radiostations',
                            'televisionstations',
                           'isps'],
             'limit': 50,
            'offset': 200,
             'radius': 40000,
             'location': 'San Diego',
            }
# Calling on the API
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

# Looping through
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

19
219


In [77]:
# Making a Lifeline 6 Dataframe
df6 = pd.DataFrame()
df6['business'] = names
df6['category'] = categories
df6['address'] = addresses
df6['lat'] = lat
df6['long'] = long
df6['lifeline'] = 6 # Creating a column for lifeline for when we merge later

In [78]:
df6['address'] = df6['address'].map(lambda x: str(x)[:-2])
df6['address'] = df6['address'].map(lambda x: str(x)[2:])
df6.head()

,business,category,address,lat,long,lifeline
0,City Captain Transportation,taxis,"3200 Adams Ave', 'Ste 105', 'San Diego, CA 92116",32.763692,-117.124766,6
1,Aall In Limo & Party Bus,limos,"655 W Broadway', 'Ste 1300', 'San Diego, CA 92101",32.715375,-117.168598,6
2,Gold Coast Town Car Services,limos,"San Diego, CA 92168",32.768551,-117.148399,6
3,Top Dog Limo Bus,limos,"1103 Emerald St', 'San Diego, CA 92109",32.799430,-117.250300,6
4,Limo Kings Limousine Service,limos,"4365 Mission Bay Dr', 'San Diego, CA 92116",32.801606,-117.158483,6


In [90]:
df6.to_csv('Lifeline_6.csv')

## Lifeline 7 - Hazardous Waste

In [79]:
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['biohazardcleanup', 'hazardouswastedisposal', 'recyclingcenter'],
             'limit': 50,
             'radius': 40000,
             'location': 'San Diego',
            }

In [80]:
# Calling on the API
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()

In [81]:
names = []       
categories = []
addresses = []
lat = []
long = []

for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

50


In [82]:
# Offset to 50
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % api_key}
parameter = {'categories': ['biohazardcleanup', 'hazardouswastedisposal', 'recyclingcenter'],
             'limit': 50,
             'offset': 50,
             'radius': 40000,
             'location': 'San Diego',
            }
# Calling on the API
req = requests.get(url=endpoint, headers=headers, params=parameter)
data = req.json()
print(len(data['businesses']))

# Looping through
for i in range(len(data['businesses'])):        
    names.append(data['businesses'][i]['name'])
    categories.append(data['businesses'][i]['categories'][0]['alias'])
    addresses.append(data['businesses'][i]['location']['display_address'])
    lat.append(data['businesses'][i]['coordinates']['latitude'])
    long.append(data['businesses'][i]['coordinates']['longitude'])

print(len(names))

3
53


In [83]:
# Making a Lifeline 7 Dataframe
df7 = pd.DataFrame()
df7['business'] = names
df7['category'] = categories
df7['address'] = addresses
df7['lat'] = lat
df7['long'] = long
df7['lifeline'] = 6 # Creating a column for lifeline for when we merge later

In [84]:
df7['address'] = df7['address'].map(lambda x: str(x)[:-2])
df7['address'] = df7['address'].map(lambda x: str(x)[2:])
df7.head()

,business,category,address,lat,long,lifeline
0,Clean Green Hauling,junkremovalandhauling,"San Diego, CA 92109",32.795220,-117.237660,6
1,Miramar Landfill,recyclingcenter,"5180 Convoy St', 'San Diego, CA 92111",32.836693,-117.152311,6
2,SES Secure E-Waste Solutions,recyclingcenter,"8810 Rehco Rd', 'Ste C', 'San Diego, CA 92121",32.882600,-117.171130,6
3,Envirogreen Recycling,recyclingcenter,"9596 Chesapeake Dr', 'Ste B', 'San Diego, CA 9...",32.834689,-117.122510,6
4,Manos Hauling and Recycling,junkremovalandhauling,"6974 Mission Gorge Rd', 'San Diego, CA 92120",32.800607,-117.091474,6


In [91]:
df7.to_csv('Lifeline_7.csv')

In [92]:
sd_df = pd.concat([df1, df2, df3, df4, df5, df6, df7],
                 axis=0)
sd_df.head()

,business,category,address,lat,long,lifeline
0,Hall of Justice,courthouses,"330 W Broadway', 'San Diego, CA 92101",32.716255,-117.166046,1
1,US Post Office,postoffices,"3288 Adams Ave', 'San Diego, CA 92116",32.763660,-117.123210,1
2,US Post Office,postoffices,"8650 Genesee Ave', 'Ste 214', 'San Diego, CA 9...",32.868689,-117.215112,1
3,US Post Office,postoffices,"11251 Rancho Carmel Dr', 'San Diego, CA 92199",32.983006,-117.081886,1
4,US Post Office,postoffices,"Macy's Home Furniture"", '1640 Camino Del Rio N...",32.769165,-117.148858,1


In [93]:
sd_df.shape

(1420, 6)

In [94]:
sd_df.to_csv('San_Diego_Lifelines.csv')